# SciPy Optimization

## Learning Objectives

By the end of this notebook, you will be able to:

1. Use `minimize` for function minimization with various algorithms
2. Find roots of equations using `root` and `brentq`
3. Handle optimization with constraints and bounds
4. Apply optimization to real-world scientific problems
5. Choose appropriate optimization methods for different problem types

---

## 1. Introduction to Optimization

Optimization is the process of finding the best solution from a set of possible solutions. In scientific computing, this often means:
- Finding minimum/maximum values of functions
- Fitting models to data
- Solving systems of equations
- Resource allocation problems

In [ ]:
import numpy as np
from scipy import optimize
from scipy.optimize import minimize, root, brentq, minimize_scalar
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Set random seed for reproducibility
np.random.seed(42)

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

print("SciPy optimization tools loaded!")

---

## 2. Scalar Minimization

For functions of a single variable, use `minimize_scalar`.

### 2.1 Basic Scalar Minimization

In [ ]:
# Define a simple function to minimize
def f(x):
    """A function with a minimum."""
    return (x - 2)**2 + 1

# Find the minimum
result = minimize_scalar(f)

print("Scalar Minimization Results")
print("=" * 35)
print(f"Minimum found at x = {result.x:.6f}")
print(f"Function value at minimum = {result.fun:.6f}")
print(f"Number of iterations = {result.nit}")
print(f"Number of function evaluations = {result.nfev}")
print(f"Success: {result.success}")

# Visualize
x = np.linspace(-2, 6, 200)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(x, f(x), 'b-', linewidth=2, label='f(x) = (x-2)² + 1')
ax.plot(result.x, result.fun, 'r*', markersize=20, label=f'Minimum at x={result.x:.2f}')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title('Scalar Function Minimization')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

### 2.2 Bounded Minimization

In [ ]:
# Function with multiple minima
def multi_minima(x):
    """Function with multiple local minima."""
    return np.sin(x) + 0.1 * x**2

# Find minimum in different regions
x = np.linspace(-10, 10, 500)

# Unbounded minimization
result_unbounded = minimize_scalar(multi_minima)

# Bounded minimization in different regions
result_left = minimize_scalar(multi_minima, bounds=(-10, -2), method='bounded')
result_center = minimize_scalar(multi_minima, bounds=(-2, 2), method='bounded')
result_right = minimize_scalar(multi_minima, bounds=(2, 10), method='bounded')

print("Bounded vs Unbounded Minimization")
print("=" * 45)
print(f"Unbounded: x = {result_unbounded.x:.4f}, f(x) = {result_unbounded.fun:.4f}")
print(f"[-10, -2]: x = {result_left.x:.4f}, f(x) = {result_left.fun:.4f}")
print(f"[-2, 2]:   x = {result_center.x:.4f}, f(x) = {result_center.fun:.4f}")
print(f"[2, 10]:   x = {result_right.x:.4f}, f(x) = {result_right.fun:.4f}")

# Visualize
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(x, multi_minima(x), 'b-', linewidth=2, label='f(x) = sin(x) + 0.1x²')
ax.plot(result_unbounded.x, result_unbounded.fun, 'r*', markersize=20, 
        label=f'Unbounded min: {result_unbounded.x:.2f}')
ax.plot(result_left.x, result_left.fun, 'go', markersize=12, 
        label=f'Left region: {result_left.x:.2f}')
ax.plot(result_center.x, result_center.fun, 'ms', markersize=12, 
        label=f'Center region: {result_center.x:.2f}')
ax.plot(result_right.x, result_right.fun, 'c^', markersize=12, 
        label=f'Right region: {result_right.x:.2f}')

# Show bounds
ax.axvline(-2, color='gray', linestyle='--', alpha=0.5)
ax.axvline(2, color='gray', linestyle='--', alpha=0.5)

ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title('Effect of Bounds on Minimization')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

---

## 3. Multivariate Minimization

The `minimize` function handles functions of multiple variables.

### 3.1 Basic Multivariate Minimization

In [ ]:
# 2D function: Rosenbrock function (a classic test function)
def rosenbrock(x):
    """Rosenbrock function: f(x,y) = (1-x)² + 100(y-x²)²
    
    Global minimum at (1, 1) with value 0.
    """
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

# Starting point
x0 = [-1, 1]

# Minimize using default method (BFGS)
result = minimize(rosenbrock, x0)

print("Rosenbrock Function Minimization")
print("=" * 40)
print(f"Starting point: {x0}")
print(f"Minimum found at: [{result.x[0]:.6f}, {result.x[1]:.6f}]")
print(f"Function value at minimum: {result.fun:.10f}")
print(f"Method used: {result.message}")
print(f"Iterations: {result.nit}")
print(f"Function evaluations: {result.nfev}")

In [ ]:
# Visualize the Rosenbrock function
x = np.linspace(-2, 2, 100)
y = np.linspace(-1, 3, 100)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100 * (Y - X**2)**2

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Contour plot
ax1 = axes[0]
contour = ax1.contour(X, Y, np.log10(Z + 1), levels=20, cmap='viridis')
ax1.plot(1, 1, 'r*', markersize=20, label='Global minimum (1, 1)')
ax1.plot(x0[0], x0[1], 'bo', markersize=10, label=f'Start point {x0}')
ax1.plot(result.x[0], result.x[1], 'gs', markersize=10, label='Found minimum')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Rosenbrock Function (log scale)')
ax1.legend()
plt.colorbar(contour, ax=ax1)

# 3D surface plot
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(X, Y, np.log10(Z + 1), cmap='viridis', alpha=0.8)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('log₁₀(f + 1)')
ax2.set_title('Rosenbrock Function 3D View')

plt.tight_layout()
plt.show()

### 3.2 Comparing Optimization Methods

In [ ]:
# Compare different optimization methods
methods = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B']

print("Comparison of Optimization Methods on Rosenbrock Function")
print("=" * 70)
print(f"{'Method':<15} {'x*':>15} {'y*':>10} {'f(x*)':>15} {'Iters':>8} {'f_evals':>8}")
print("-" * 70)

results_dict = {}

for method in methods:
    result = minimize(rosenbrock, x0, method=method)
    results_dict[method] = result
    print(f"{method:<15} {result.x[0]:>15.6f} {result.x[1]:>10.6f} "
          f"{result.fun:>15.2e} {result.nit:>8} {result.nfev:>8}")

### 3.3 Providing Gradients

In [ ]:
# Define gradient (Jacobian) for Rosenbrock function
def rosenbrock_gradient(x):
    """Gradient of Rosenbrock function."""
    dfdx = -2 * (1 - x[0]) - 400 * x[0] * (x[1] - x[0]**2)
    dfdy = 200 * (x[1] - x[0]**2)
    return np.array([dfdx, dfdy])

# Compare with and without gradient
result_no_grad = minimize(rosenbrock, x0, method='BFGS')
result_with_grad = minimize(rosenbrock, x0, method='BFGS', jac=rosenbrock_gradient)

print("Effect of Providing Gradient")
print("=" * 50)
print(f"{'':20} {'Without Gradient':>15} {'With Gradient':>15}")
print("-" * 50)
print(f"{'Function evaluations':<20} {result_no_grad.nfev:>15} {result_with_grad.nfev:>15}")
print(f"{'Gradient evaluations':<20} {'(approx)':>15} {result_with_grad.njev:>15}")
print(f"{'Iterations':<20} {result_no_grad.nit:>15} {result_with_grad.nit:>15}")
print(f"{'Final value':<20} {result_no_grad.fun:>15.2e} {result_with_grad.fun:>15.2e}")

---

## 4. Constrained Optimization

Many real-world problems have constraints that limit the feasible solutions.

### 4.1 Bound Constraints

In [ ]:
# Simple quadratic function
def quadratic(x):
    return x[0]**2 + x[1]**2

# Bounds: 1 <= x <= 3, 2 <= y <= 4
bounds = [(1, 3), (2, 4)]

# Minimize with bounds
result_bounded = minimize(quadratic, [2, 3], bounds=bounds, method='L-BFGS-B')
result_unbounded = minimize(quadratic, [2, 3], method='BFGS')

print("Bounded vs Unbounded Optimization")
print("=" * 45)
print(f"Unbounded minimum: ({result_unbounded.x[0]:.4f}, {result_unbounded.x[1]:.4f})")
print(f"Unbounded f(x): {result_unbounded.fun:.4f}")
print(f"\nBounded minimum: ({result_bounded.x[0]:.4f}, {result_bounded.x[1]:.4f})")
print(f"Bounded f(x): {result_bounded.fun:.4f}")
print(f"Bounds: x ∈ [1, 3], y ∈ [2, 4]")

# Visualize
x = np.linspace(-1, 5, 100)
y = np.linspace(-1, 5, 100)
X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

fig, ax = plt.subplots(figsize=(8, 6))
contour = ax.contour(X, Y, Z, levels=20, cmap='viridis')

# Draw bounds
rect = plt.Rectangle((1, 2), 2, 2, fill=False, edgecolor='red', linewidth=2, 
                      linestyle='--', label='Feasible region')
ax.add_patch(rect)

ax.plot(0, 0, 'b*', markersize=20, label='Unbounded minimum')
ax.plot(result_bounded.x[0], result_bounded.x[1], 'r*', markersize=20, 
        label='Bounded minimum')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Optimization with Bound Constraints')
ax.legend()
plt.colorbar(contour, ax=ax)
plt.show()

### 4.2 Linear and Nonlinear Constraints

In [ ]:
# Minimize f(x,y) = x² + y² subject to constraints
def objective(x):
    return x[0]**2 + x[1]**2

# Constraint 1: x + y >= 1 (linear inequality)
# Written as: 1 - x - y <= 0 (scipy format: constraint(x) >= 0 for ineq)
def constraint1(x):
    return x[0] + x[1] - 1  # x + y - 1 >= 0

# Constraint 2: x² + y² <= 4 (nonlinear inequality)
# Written as: 4 - x² - y² >= 0
def constraint2(x):
    return 4 - x[0]**2 - x[1]**2  # Must be >= 0

# Define constraints for scipy
constraints = [
    {'type': 'ineq', 'fun': constraint1},  # x + y >= 1
    {'type': 'ineq', 'fun': constraint2},  # x² + y² <= 4
]

# Initial guess
x0 = [1, 1]

# Minimize with constraints using SLSQP
result = minimize(objective, x0, method='SLSQP', constraints=constraints)

print("Constrained Optimization Results")
print("=" * 45)
print(f"Optimal point: ({result.x[0]:.6f}, {result.x[1]:.6f})")
print(f"Objective value: {result.fun:.6f}")
print(f"\nConstraint satisfaction:")
print(f"  x + y = {result.x[0] + result.x[1]:.4f} (should be >= 1)")
print(f"  x² + y² = {result.x[0]**2 + result.x[1]**2:.4f} (should be <= 4)")

In [ ]:
# Visualize constrained optimization
x = np.linspace(-3, 3, 200)
y = np.linspace(-3, 3, 200)
X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

fig, ax = plt.subplots(figsize=(8, 8))

# Objective function contours
contour = ax.contour(X, Y, Z, levels=15, cmap='viridis', alpha=0.7)
plt.colorbar(contour, ax=ax, label='f(x,y) = x² + y²')

# Constraint 1: x + y >= 1 (shade infeasible region)
ax.fill_between(x, -3, 1 - x, alpha=0.3, color='red', label='x + y < 1 (infeasible)')

# Constraint 2: x² + y² <= 4 (circle)
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(2*np.cos(theta), 2*np.sin(theta), 'b--', linewidth=2, label='x² + y² = 4')

# Optimal point
ax.plot(result.x[0], result.x[1], 'r*', markersize=20, label=f'Optimal: ({result.x[0]:.2f}, {result.x[1]:.2f})')

ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Constrained Optimization')
ax.legend(loc='upper right')
ax.set_aspect('equal')
plt.show()

### 4.3 Equality Constraints

In [ ]:
# Minimize distance from origin, constrained to a circle
# Minimize: f(x,y) = (x-3)² + (y-3)²
# Subject to: x² + y² = 4

def distance_objective(x):
    return (x[0] - 3)**2 + (x[1] - 3)**2

def circle_constraint(x):
    return x[0]**2 + x[1]**2 - 4  # Should equal 0

constraints_eq = [
    {'type': 'eq', 'fun': circle_constraint}
]

# Different starting points
starting_points = [[1, 1], [-1, 1], [0, 2]]

print("Optimization with Equality Constraint")
print("=" * 50)
print("Find point on circle x² + y² = 4 closest to (3, 3)")
print("\nFrom different starting points:")

for x0 in starting_points:
    result = minimize(distance_objective, x0, method='SLSQP', constraints=constraints_eq)
    print(f"  Start {x0} -> ({result.x[0]:.4f}, {result.x[1]:.4f}), "
          f"dist = {np.sqrt(result.fun):.4f}")

# Visualize
fig, ax = plt.subplots(figsize=(8, 8))

# Circle constraint
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(2*np.cos(theta), 2*np.sin(theta), 'b-', linewidth=2, label='Constraint: x² + y² = 4')

# Target point
ax.plot(3, 3, 'g*', markersize=20, label='Target (3, 3)')

# Optimal point (should be at 45°)
result = minimize(distance_objective, [1, 1], method='SLSQP', constraints=constraints_eq)
ax.plot(result.x[0], result.x[1], 'r*', markersize=20, 
        label=f'Closest point ({result.x[0]:.2f}, {result.x[1]:.2f})')

# Line from target to closest point
ax.plot([3, result.x[0]], [3, result.x[1]], 'r--', linewidth=2)

ax.set_xlim(-3, 4)
ax.set_ylim(-3, 4)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Optimization on a Constraint Surface')
ax.legend()
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
plt.show()

---

## 5. Root Finding

Finding roots (zeros) of equations is another fundamental optimization task.

### 5.1 Scalar Root Finding

In [ ]:
# Find root of a nonlinear equation
def equation(x):
    return x**3 - 2*x - 5

# Method 1: brentq (requires bracketing interval)
root_brentq = brentq(equation, 1, 3)  # Root must be between 1 and 3

# Method 2: brent (doesn't require strict bracketing)
from scipy.optimize import brent
root_brent = brent(lambda x: equation(x)**2, brack=(1, 3))  # Minimize squared equation

# Method 3: newton (Newton-Raphson method)
from scipy.optimize import newton
root_newton = newton(equation, x0=2)

print("Root Finding for x³ - 2x - 5 = 0")
print("=" * 40)
print(f"brentq method: x = {root_brentq:.10f}")
print(f"newton method: x = {root_newton:.10f}")
print(f"\nVerification: f({root_brentq:.6f}) = {equation(root_brentq):.2e}")

# Visualize
x = np.linspace(0, 3, 200)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(x, equation(x), 'b-', linewidth=2, label='f(x) = x³ - 2x - 5')
ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.plot(root_brentq, 0, 'r*', markersize=20, label=f'Root: x = {root_brentq:.4f}')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title('Root Finding')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

### 5.2 Systems of Equations

In [ ]:
# Solve system of nonlinear equations:
# x² + y² = 10
# x*y = 3

def system(vars):
    x, y = vars
    eq1 = x**2 + y**2 - 10  # Should be 0
    eq2 = x * y - 3         # Should be 0
    return [eq1, eq2]

# Initial guess
x0 = [1, 3]

# Solve using root
solution = root(system, x0)

print("System of Nonlinear Equations")
print("=" * 45)
print("Equations: x² + y² = 10, xy = 3")
print(f"\nSolution: x = {solution.x[0]:.6f}, y = {solution.x[1]:.6f}")
print(f"\nVerification:")
print(f"  x² + y² = {solution.x[0]**2 + solution.x[1]**2:.6f} (should be 10)")
print(f"  x*y = {solution.x[0] * solution.x[1]:.6f} (should be 3)")
print(f"\nSuccess: {solution.success}")

In [ ]:
# Visualize the solution
fig, ax = plt.subplots(figsize=(8, 8))

# Circle: x² + y² = 10
theta = np.linspace(0, 2*np.pi, 100)
r = np.sqrt(10)
ax.plot(r*np.cos(theta), r*np.sin(theta), 'b-', linewidth=2, label='x² + y² = 10')

# Hyperbola: xy = 3
x_pos = np.linspace(0.3, 4, 100)
x_neg = np.linspace(-4, -0.3, 100)
ax.plot(x_pos, 3/x_pos, 'g-', linewidth=2, label='xy = 3')
ax.plot(x_neg, 3/x_neg, 'g-', linewidth=2)

# Find all solutions (multiple initial guesses)
initial_guesses = [[1, 3], [3, 1], [-1, -3], [-3, -1]]
solutions_found = []

for guess in initial_guesses:
    sol = root(system, guess)
    if sol.success:
        # Check if this solution is unique (not already found)
        is_new = True
        for prev_sol in solutions_found:
            if np.allclose(sol.x, prev_sol, atol=1e-5):
                is_new = False
                break
        if is_new:
            solutions_found.append(sol.x)

# Plot all solutions
for i, sol in enumerate(solutions_found):
    ax.plot(sol[0], sol[1], 'r*', markersize=20, 
            label=f'Solution {i+1}: ({sol[0]:.2f}, {sol[1]:.2f})' if i == 0 else f'({sol[0]:.2f}, {sol[1]:.2f})')

ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('System of Nonlinear Equations: Intersection Points')
ax.legend()
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
plt.show()

print(f"\nFound {len(solutions_found)} solution(s)")

---

## 6. Practical Examples

### 6.1 Portfolio Optimization

In [ ]:
# Simple portfolio optimization: minimize risk for target return
# Assets: Stock A, Stock B, Bond

# Expected returns (annual)
returns = np.array([0.12, 0.08, 0.04])  # 12%, 8%, 4%

# Covariance matrix (risk relationships)
cov_matrix = np.array([
    [0.04, 0.006, 0.001],   # Stock A variance and covariances
    [0.006, 0.025, 0.002],  # Stock B variance and covariances
    [0.001, 0.002, 0.01]    # Bond variance and covariances
])

def portfolio_risk(weights):
    """Portfolio variance (risk squared)."""
    return weights.T @ cov_matrix @ weights

def portfolio_return(weights):
    """Expected portfolio return."""
    return np.sum(returns * weights)

# Target return: 8%
target_return = 0.08

# Constraints:
# 1. Weights sum to 1
# 2. Portfolio return equals target
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
    {'type': 'eq', 'fun': lambda w: portfolio_return(w) - target_return}
]

# Bounds: no short selling (weights >= 0)
bounds = [(0, 1) for _ in range(3)]

# Initial guess: equal weights
w0 = np.array([1/3, 1/3, 1/3])

# Optimize
result = minimize(portfolio_risk, w0, method='SLSQP', 
                  bounds=bounds, constraints=constraints)

print("Portfolio Optimization Results")
print("=" * 45)
print(f"Target return: {target_return*100:.1f}%")
print(f"\nOptimal Allocation:")
print(f"  Stock A: {result.x[0]*100:.1f}%")
print(f"  Stock B: {result.x[1]*100:.1f}%")
print(f"  Bond:    {result.x[2]*100:.1f}%")
print(f"\nPortfolio Statistics:")
print(f"  Expected Return: {portfolio_return(result.x)*100:.2f}%")
print(f"  Risk (Std Dev): {np.sqrt(result.fun)*100:.2f}%")

In [ ]:
# Plot efficient frontier
target_returns = np.linspace(0.04, 0.12, 50)
risks = []
allocations = []

for target in target_returns:
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        {'type': 'eq', 'fun': lambda w, t=target: portfolio_return(w) - t}
    ]
    result = minimize(portfolio_risk, w0, method='SLSQP', 
                      bounds=bounds, constraints=constraints)
    if result.success:
        risks.append(np.sqrt(result.fun) * 100)
        allocations.append(result.x)
    else:
        risks.append(np.nan)
        allocations.append([np.nan, np.nan, np.nan])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Efficient frontier
ax1 = axes[0]
ax1.plot(risks, target_returns*100, 'b-', linewidth=2, label='Efficient Frontier')
ax1.scatter([np.sqrt(cov_matrix[i,i])*100 for i in range(3)], 
            returns*100, s=100, c=['red', 'green', 'orange'], 
            marker='*', zorder=5, label='Individual Assets')
ax1.annotate('Stock A', (np.sqrt(cov_matrix[0,0])*100 + 0.5, returns[0]*100))
ax1.annotate('Stock B', (np.sqrt(cov_matrix[1,1])*100 + 0.5, returns[1]*100))
ax1.annotate('Bond', (np.sqrt(cov_matrix[2,2])*100 + 0.5, returns[2]*100))
ax1.set_xlabel('Risk (Standard Deviation %)')
ax1.set_ylabel('Expected Return %')
ax1.set_title('Efficient Frontier')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Allocation vs return
allocations = np.array(allocations)
ax2 = axes[1]
ax2.stackplot(target_returns*100, allocations.T * 100, 
              labels=['Stock A', 'Stock B', 'Bond'],
              colors=['red', 'green', 'orange'], alpha=0.7)
ax2.set_xlabel('Target Return %')
ax2.set_ylabel('Allocation %')
ax2.set_title('Optimal Asset Allocation')
ax2.legend(loc='center left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 6.2 Chemical Equilibrium

In [ ]:
# Chemical equilibrium: 2A ⇌ B + C
# Equilibrium constant K = [B][C] / [A]²
# Given initial concentration of A, find equilibrium concentrations

def equilibrium_equations(x, A0, K):
    """
    x = extent of reaction
    At equilibrium:
    [A] = A0 - 2x
    [B] = x
    [C] = x
    """
    A = A0 - 2*x
    B = x
    C = x
    # K = [B][C] / [A]²
    return B * C / A**2 - K

# Parameters
A0 = 1.0  # Initial concentration of A
K = 4.0   # Equilibrium constant

# Find extent of reaction (must be between 0 and A0/2)
x_eq = brentq(equilibrium_equations, 0.001, A0/2 - 0.001, args=(A0, K))

# Calculate equilibrium concentrations
A_eq = A0 - 2*x_eq
B_eq = x_eq
C_eq = x_eq

print("Chemical Equilibrium: 2A ⇌ B + C")
print("=" * 40)
print(f"Initial [A]: {A0} M")
print(f"Equilibrium constant K: {K}")
print(f"\nEquilibrium Concentrations:")
print(f"  [A] = {A_eq:.4f} M")
print(f"  [B] = {B_eq:.4f} M")
print(f"  [C] = {C_eq:.4f} M")
print(f"\nVerification:")
print(f"  K = [B][C]/[A]² = {B_eq * C_eq / A_eq**2:.4f}")
print(f"  Mass balance: 2×[A] + [B] + [C] = {2*A_eq + B_eq + C_eq:.4f} (should = {2*A0})")

### 6.3 Least Squares Fitting

In [ ]:
# Custom least squares fitting using optimization
# Fit: y = a * sin(b*x + c) + d

# Generate noisy data
np.random.seed(42)
x_data = np.linspace(0, 4*np.pi, 50)
a_true, b_true, c_true, d_true = 2.5, 1.5, 0.5, 1.0
y_true = a_true * np.sin(b_true * x_data + c_true) + d_true
y_data = y_true + np.random.normal(0, 0.3, len(x_data))

def sinusoidal(params, x):
    a, b, c, d = params
    return a * np.sin(b * x + c) + d

def residuals(params, x, y):
    """Sum of squared residuals."""
    return np.sum((y - sinusoidal(params, x))**2)

# Initial guess
params0 = [2, 1, 0, 0]

# Minimize
result = minimize(residuals, params0, args=(x_data, y_data), method='Nelder-Mead')
params_fit = result.x

print("Sinusoidal Fit: y = a*sin(b*x + c) + d")
print("=" * 45)
print(f"{'Parameter':<10} {'True':>10} {'Fitted':>10}")
print("-" * 30)
print(f"{'a':<10} {a_true:>10.4f} {params_fit[0]:>10.4f}")
print(f"{'b':<10} {b_true:>10.4f} {params_fit[1]:>10.4f}")
print(f"{'c':<10} {c_true:>10.4f} {params_fit[2]:>10.4f}")
print(f"{'d':<10} {d_true:>10.4f} {params_fit[3]:>10.4f}")

# Plot
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(x_data, y_data, 'ko', alpha=0.5, label='Data')
x_plot = np.linspace(0, 4*np.pi, 200)
ax.plot(x_plot, sinusoidal(params_fit, x_plot), 'r-', linewidth=2, label='Fitted')
ax.plot(x_plot, sinusoidal([a_true, b_true, c_true, d_true], x_plot), 
        'g--', linewidth=2, alpha=0.7, label='True')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Sinusoidal Curve Fitting via Optimization')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

---

## Exercises

Practice what you've learned with these exercises.

### Exercise 1: Finding Extrema

Find all local minima and maxima of f(x) = x⁴ - 4x³ + 4x² on the interval [-1, 3].

1. Plot the function to visualize extrema
2. Use minimize_scalar to find minima in different regions
3. Find maxima by minimizing -f(x)
4. Verify results analytically (f'(x) = 0)

In [ ]:
def f(x):
    return x**4 - 4*x**3 + 4*x**2

# Your code here


<details>
<summary>Click to see solution</summary>

```python
# 1. Plot the function
x_plot = np.linspace(-1, 3, 200)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(x_plot, f(x_plot), 'b-', linewidth=2, label='f(x) = x⁴ - 4x³ + 4x²')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.grid(True, alpha=0.3)

# 2. Find minima in different regions
min1 = minimize_scalar(f, bounds=(-1, 1), method='bounded')
min2 = minimize_scalar(f, bounds=(1, 3), method='bounded')

# 3. Find maxima (minimize -f)
max1 = minimize_scalar(lambda x: -f(x), bounds=(0, 2), method='bounded')

print("Extrema of f(x) = x⁴ - 4x³ + 4x²")
print("=" * 40)
print(f"\nLocal minimum 1: x = {min1.x:.4f}, f(x) = {min1.fun:.4f}")
print(f"Local minimum 2: x = {min2.x:.4f}, f(x) = {min2.fun:.4f}")
print(f"Local maximum:   x = {max1.x:.4f}, f(x) = {-max1.fun:.4f}")

# Plot extrema
ax.plot(min1.x, min1.fun, 'go', markersize=12, label=f'Min: ({min1.x:.2f}, {min1.fun:.2f})')
ax.plot(min2.x, min2.fun, 'go', markersize=12, label=f'Min: ({min2.x:.2f}, {min2.fun:.2f})')
ax.plot(max1.x, -max1.fun, 'r^', markersize=12, label=f'Max: ({max1.x:.2f}, {-max1.fun:.2f})')
ax.legend()
ax.set_title('Finding Extrema')
plt.show()

# 4. Analytical verification
# f'(x) = 4x³ - 12x² + 8x = 4x(x² - 3x + 2) = 4x(x-1)(x-2)
# Critical points: x = 0, 1, 2
print("\nAnalytical verification (f'(x) = 4x(x-1)(x-2) = 0):")
print(f"Critical points: x = 0, 1, 2")
print(f"f(0) = {f(0)}, f(1) = {f(1)}, f(2) = {f(2)}")
```
</details>

### Exercise 2: Constrained Optimization Problem

A farmer has 100 meters of fencing and wants to enclose a rectangular area next to a river (no fence needed on the river side). Find the dimensions that maximize the area.

Let x = length parallel to river, y = width perpendicular to river.
- Maximize: A = x * y
- Subject to: x + 2y = 100 (fencing constraint)
- And: x, y > 0

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
# Objective: minimize negative area (to maximize area)
def neg_area(vars):
    x, y = vars
    return -x * y

# Constraint: x + 2y = 100
def fencing_constraint(vars):
    x, y = vars
    return x + 2*y - 100

constraints = [
    {'type': 'eq', 'fun': fencing_constraint}
]

# Bounds: x, y > 0 (and practical upper limits)
bounds = [(0.1, 100), (0.1, 50)]

# Initial guess
x0 = [40, 30]

# Optimize
result = minimize(neg_area, x0, method='SLSQP', 
                  bounds=bounds, constraints=constraints)

x_opt, y_opt = result.x
area_opt = -result.fun

print("Farmer's Fencing Problem")
print("=" * 40)
print(f"Optimal dimensions:")
print(f"  Length (along river): {x_opt:.2f} meters")
print(f"  Width (perpendicular): {y_opt:.2f} meters")
print(f"\nMaximum area: {area_opt:.2f} square meters")
print(f"\nVerification:")
print(f"  Fencing used: {x_opt + 2*y_opt:.2f} meters (should = 100)")

# Analytical solution: A = x*y with x = 100 - 2y
# A = (100 - 2y)*y = 100y - 2y²
# dA/dy = 100 - 4y = 0 => y = 25, x = 50
print("\nAnalytical solution: x = 50, y = 25, A = 1250")

# Visualize
y_range = np.linspace(1, 49, 100)
x_range = 100 - 2*y_range
area_range = x_range * y_range

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(y_range, area_range, 'b-', linewidth=2, label='Area vs Width')
ax.plot(y_opt, area_opt, 'r*', markersize=20, label=f'Max area: {area_opt:.0f} m²')
ax.set_xlabel('Width y (meters)')
ax.set_ylabel('Area (square meters)')
ax.set_title('Area as Function of Width')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()
```
</details>

### Exercise 3: Root Finding Application

The van der Waals equation for real gases is:
(P + a/V²)(V - b) = RT

For CO2: a = 3.59 L²·bar/mol², b = 0.0427 L/mol
R = 0.08314 L·bar/(mol·K)

Find the molar volume V at T = 300K and P = 10 bar.

In [ ]:
# Van der Waals parameters for CO2
a = 3.59    # L²·bar/mol²
b = 0.0427  # L/mol
R = 0.08314 # L·bar/(mol·K)
T = 300     # K
P = 10      # bar

# Your code here


<details>
<summary>Click to see solution</summary>

```python
# Van der Waals equation rearranged: (P + a/V²)(V - b) - RT = 0
def van_der_waals(V, P, T, a, b, R):
    return (P + a/V**2) * (V - b) - R*T

# Ideal gas volume as initial estimate: V = RT/P
V_ideal = R * T / P
print(f"Ideal gas molar volume: {V_ideal:.4f} L/mol")

# Find root using brentq
# V must be > b (about 0.04) and we expect it to be around 2-3 L/mol
V_vdw = brentq(van_der_waals, b + 0.01, 10, args=(P, T, a, b, R))

print("\nVan der Waals Equation Solution")
print("=" * 45)
print(f"Temperature: {T} K")
print(f"Pressure: {P} bar")
print(f"\nIdeal gas V: {V_ideal:.4f} L/mol")
print(f"Van der Waals V: {V_vdw:.4f} L/mol")
print(f"Deviation: {(V_ideal - V_vdw)/V_vdw * 100:.2f}%")

# Verification
print(f"\nVerification:")
lhs = (P + a/V_vdw**2) * (V_vdw - b)
rhs = R * T
print(f"  LHS = (P + a/V²)(V - b) = {lhs:.6f}")
print(f"  RHS = RT = {rhs:.6f}")

# Plot to visualize
V_range = np.linspace(b + 0.1, 5, 200)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(V_range, van_der_waals(V_range, P, T, a, b, R), 'b-', linewidth=2, 
        label='Van der Waals equation')
ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.plot(V_vdw, 0, 'r*', markersize=20, label=f'V = {V_vdw:.3f} L/mol')
ax.plot(V_ideal, van_der_waals(V_ideal, P, T, a, b, R), 'go', markersize=10,
        label=f'Ideal gas V = {V_ideal:.3f} L/mol')
ax.set_xlabel('Molar Volume V (L/mol)')
ax.set_ylabel('f(V)')
ax.set_title('Van der Waals Equation Root Finding')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()
```
</details>

### Exercise 4: Multidimensional Optimization

Find the minimum of the Himmelblau function:
f(x,y) = (x² + y - 11)² + (x + y² - 7)²

This function has multiple local minima. Find all of them.

In [ ]:
def himmelblau(x):
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2

# Your code here


<details>
<summary>Click to see solution</summary>

```python
# Try multiple starting points
starting_points = [
    [0, 0], [4, 4], [-4, 4], [4, -4], [-4, -4],
    [2, 2], [-2, 2], [2, -2], [-2, -2]
]

minima = []
print("Himmelblau Function Minima")
print("=" * 50)

for x0 in starting_points:
    result = minimize(himmelblau, x0, method='BFGS')
    if result.success and result.fun < 1e-6:  # Found a true minimum
        # Check if this is a new minimum
        is_new = True
        for prev in minima:
            if np.allclose(result.x, prev, atol=0.1):
                is_new = False
                break
        if is_new:
            minima.append(result.x)

print(f"Found {len(minima)} local minima:")
for i, m in enumerate(minima):
    print(f"  Minimum {i+1}: ({m[0]:.4f}, {m[1]:.4f}), f = {himmelblau(m):.2e}")

# Create contour plot
x = np.linspace(-6, 6, 200)
y = np.linspace(-6, 6, 200)
X, Y = np.meshgrid(x, y)
Z = (X**2 + Y - 11)**2 + (X + Y**2 - 7)**2

fig, ax = plt.subplots(figsize=(10, 8))
contour = ax.contour(X, Y, np.log10(Z + 1), levels=30, cmap='viridis')
plt.colorbar(contour, ax=ax, label='log₁₀(f + 1)')

# Mark all minima
colors = ['red', 'blue', 'green', 'orange']
for i, m in enumerate(minima):
    ax.plot(m[0], m[1], '*', markersize=20, color=colors[i % len(colors)],
            label=f'Min {i+1}: ({m[0]:.2f}, {m[1]:.2f})')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Himmelblau Function - Multiple Local Minima')
ax.legend(loc='upper left')
ax.set_aspect('equal')
plt.show()

print("\nKnown analytical minima:")
print("(3.0, 2.0), (-2.805, 3.131), (-3.779, -3.283), (3.584, -1.848)")
```
</details>

### Exercise 5: Practical Problem - Projectile Motion

A projectile is launched with initial speed v0 = 20 m/s. Find the launch angle θ that maximizes the range on flat ground.

Range formula: R = (v0² × sin(2θ)) / g

1. Use optimization to find the optimal angle
2. Verify the analytical solution (θ = 45°)
3. Plot range vs angle

In [ ]:
v0 = 20  # m/s
g = 9.81 # m/s²

# Your code here


<details>
<summary>Click to see solution</summary>

```python
def range_formula(theta):
    """Calculate projectile range. Theta in radians."""
    return (v0**2 * np.sin(2 * theta)) / g

def neg_range(theta):
    """Negative range for minimization."""
    return -range_formula(theta)

# 1. Find optimal angle (must be between 0 and π/2)
result = minimize_scalar(neg_range, bounds=(0, np.pi/2), method='bounded')
theta_opt = result.x
range_max = -result.fun

print("Projectile Motion Optimization")
print("=" * 40)
print(f"Initial velocity: {v0} m/s")
print(f"Gravitational acceleration: {g} m/s²")
print(f"\nOptimal launch angle: {np.degrees(theta_opt):.2f}°")
print(f"Maximum range: {range_max:.2f} m")

# 2. Analytical solution
print(f"\nAnalytical solution: θ = 45°")
print(f"Maximum range formula: R_max = v0²/g = {v0**2/g:.2f} m")

# 3. Plot range vs angle
theta_range = np.linspace(0, np.pi/2, 100)
ranges = range_formula(theta_range)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Range vs angle plot
ax1 = axes[0]
ax1.plot(np.degrees(theta_range), ranges, 'b-', linewidth=2, label='Range')
ax1.plot(np.degrees(theta_opt), range_max, 'r*', markersize=20, 
         label=f'Max: {range_max:.1f}m at {np.degrees(theta_opt):.1f}°')
ax1.set_xlabel('Launch Angle (degrees)')
ax1.set_ylabel('Range (m)')
ax1.set_title('Projectile Range vs Launch Angle')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Trajectory plot
ax2 = axes[1]
t_max = 2 * v0 * np.sin(theta_opt) / g
t = np.linspace(0, t_max, 100)

# Trajectories at different angles
for angle in [30, 45, 60, 75]:
    theta = np.radians(angle)
    t_flight = 2 * v0 * np.sin(theta) / g
    t = np.linspace(0, t_flight, 100)
    x = v0 * np.cos(theta) * t
    y = v0 * np.sin(theta) * t - 0.5 * g * t**2
    style = 'r-' if angle == 45 else 'b--'
    lw = 3 if angle == 45 else 1
    ax2.plot(x, y, style, linewidth=lw, label=f'{angle}°')

ax2.set_xlabel('Horizontal Distance (m)')
ax2.set_ylabel('Height (m)')
ax2.set_title('Projectile Trajectories')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, None)

plt.tight_layout()
plt.show()
```
</details>

---

## Summary

In this notebook, you learned:

1. **Scalar Minimization**
   - `minimize_scalar` for 1D optimization
   - Bounded vs unbounded optimization

2. **Multivariate Minimization**
   - `minimize` function with various methods
   - Providing gradients for faster convergence
   - Comparison of optimization methods

3. **Constrained Optimization**
   - Bound constraints with `bounds` parameter
   - Equality and inequality constraints
   - SLSQP method for constrained problems

4. **Root Finding**
   - `brentq` for bracketed scalar equations
   - `root` for systems of equations
   - Multiple solutions from different starting points

5. **Practical Applications**
   - Portfolio optimization
   - Chemical equilibrium
   - Curve fitting

---

## Next Steps

Continue your SciPy journey with the next notebook:

**[04_integration_odes.ipynb](04_integration_odes.ipynb)** - Learn about numerical integration and solving ordinary differential equations (ODEs).